In [2]:
!git init
!git remote add origin https://github.com/lorenzobagnol/bgl_minGPT.git
!git pull origin main

Reinitialized existing Git repository in C:/Users/loren/Documents/Lorenzo/UniPi/minGPT/.git/


error: remote origin already exists.
From https://github.com/lorenzobagnol/bgl_minGPT
 * branch            main       -> FETCH_HEAD


Already up to date.


In [3]:
%pip install -r requirements.txt

You should consider upgrading via the 'c:\Users\loren\Documents\Lorenzo\UniPi\minGPT\minGPT_Env\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
from mingpt.model import GPT
import json

'''using a gpt2 vocab and tokenizer with vocabulary dimension of 50257'''
with open('./gpt_experiment/gpt2_vocab.json',encoding="ascii",errors=" ") as json_file: 
    data = json.load(json_file)
model_config = GPT.get_default_config()
model_config.model_type='gpt-nano'
model_config.vocab_size = len(data)
model_config.block_size =48

In [5]:
model=GPT(model_config)

number of parameters: 2.50M


In [6]:
# first try run without any parameter optimization
import torch
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenized_input=torch.tensor([tokenizer("Vice distortions")['input_ids']], dtype=torch.long)
with torch.no_grad():
    cat = model.generate(tokenized_input, 4, do_sample=False)
cat


tensor([[47910, 47876, 18260,  4753, 26840, 36749]])

In [7]:
# clean and tokenize every sentence in the dataset
import random
import requests, json
from torch.utils.data import Dataset

class SentenceDataset(Dataset):

    def __init__(self, split, length=48):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        API_URL =  "https://datasets-server.huggingface.co/first-rows?dataset=bookcorpus&config=plain_text&split=train"
        def query():
            response = requests.request("GET", API_URL)
            return response.json()
        data = query()
        tokenized_sentences=[]
        sentences=data['rows']
        for i in range(100):
            tokenized_sen=torch.tensor([tokenizer(sentences[i]['row']['text'])['input_ids']], dtype=torch.long)
            if(len(tokenized_sen)>length):
                tokenized_sen=tokenized_sen[:][:length]
            tokenized_sentences.append(tokenized_sen)
        self.samples=tokenized_sentences
    
    def __len__(self):
        return 100
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length #* 2-1

    def __getitem__(self, idx):
        sentence=self.samples[idx][0]
        residual_length=192-len(sentence)
        masked=torch.cat((torch.tensor(sentence[:-1]),torch.tensor([50256]*(residual_length+1))),0)
        sol=torch.cat((sentence,torch.tensor([50256]*(residual_length))),0)
        sol[0]=50256
        return masked, sol
        #return sentence, sol


In [8]:
with open("./books.txt",encoding="utf16") as file:
    tokenized_corpus=tokenizer(file.read())['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (4793788 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
import os

class BookDataset(Dataset):

    def __init__(self, split, length=48):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        tokenized_sentences=[]
        with open("./books.txt",encoding="utf16") as file:
            tokenized_corpus=torch.tensor(tokenizer(file.read())['input_ids'], dtype=torch.long)
        corpus_len=len(tokenized_corpus)-(len(tokenized_corpus)%length)
        tokenized_corpus=tokenized_corpus[:corpus_len]
        tokenized_sentences=[]
        self.len=int(corpus_len/length)
        for i in range (self.len):
            tokenized_sentences.append(tokenized_corpus[i*length:(i+1)*length])
        self.samples=tokenized_sentences
    
    def __len__(self):
        return self.len #number of sentences
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length #* 2-1

    def __getitem__(self, idx):
        sentence=self.samples[idx]
        input= sentence
        output=torch.cat((torch.tensor(sentence[1:]),torch.tensor([-1])),0)
        return input, output
        #return sentence, sol


In [10]:
train_dataset = BookDataset('train')
test_dataset = BookDataset('test')

In [11]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config() # we can also change some of them if we want
train_config.learning_rate = 5e-4 
train_config.max_iters = 2000
train_config.num_workers = 0
train_config.batch_size=20
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [14]:
checkpoint = torch.load('model.pt',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
from torch.utils.tensorboard import SummaryWriter

tb = SummaryWriter()
def batch_end_callback(trainer):
    if trainer.iter_num % 10 == 0:
        
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
        tb.add_scalar("Loss", trainer.loss.item(), trainer.iter_num)
tb.close()
trainer.set_callback('on_batch_end', batch_end_callback)
train_config.learning_rate=5e-6
trainer.run()

C:\Users\loren\AppData\Local\Temp\ipykernel_29912\802256824.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output=torch.cat((torch.tensor(sentence[1:]),torch.tensor([-1])),0)


iter_dt 2236.02ms; iter 0: train loss 4.45012
iter_dt 1426.88ms; iter 10: train loss 5.05880
iter_dt 1544.50ms; iter 20: train loss 4.91202
iter_dt 1406.41ms; iter 30: train loss 4.86848
iter_dt 1500.83ms; iter 40: train loss 4.88711
iter_dt 1597.69ms; iter 50: train loss 4.70363
iter_dt 1469.77ms; iter 60: train loss 4.73650
iter_dt 1568.50ms; iter 70: train loss 4.69944
iter_dt 1529.03ms; iter 80: train loss 4.98098
iter_dt 1588.93ms; iter 90: train loss 4.76855
iter_dt 1552.70ms; iter 100: train loss 4.79214
iter_dt 1562.43ms; iter 110: train loss 4.64490
iter_dt 1626.03ms; iter 120: train loss 4.72393
iter_dt 2207.46ms; iter 130: train loss 4.71895
iter_dt 1481.83ms; iter 140: train loss 4.62808
iter_dt 1481.83ms; iter 150: train loss 4.94885
iter_dt 1443.19ms; iter 160: train loss 4.57405
iter_dt 2164.18ms; iter 170: train loss 5.03615


KeyboardInterrupt: 

In [ ]:
optimizer = model.configure_optimizers(train_config)
PATH = "model.pt"
torch.save({
    'model_state_dict':model.state_dict()
}, PATH)

In [ ]:
for param in model.parameters():
    print(param)

In [25]:
inputs = tokenizer("Hello,", return_tensors="pt")['input_ids']
output=model.generate(inputs,20)
inverted_vocab= {b:a for a, b in data.items()}
out_string=""
for i in range(20):
      out_string=out_string+inverted_vocab[output[0,i].item()]
print(out_string.replace("Ġ"," ").replace("Ċ"," "))

Hello, I'm not a the time to be a few minutes.  "I'm
